In [1]:
import pandas as pd
import numpy as np
import ffmpy
import os
import glob
from datetime import datetime
from os import listdir
from os.path import isfile, join

In [2]:
#Excel sheet which contains FACS ENCODING FOR ALL THE VIDEO'S
df1 = pd.read_csv("C:/Users/Vijay/Isaac/Issac Video/Working_code_on_training_vedios/purl2_coding/purl2_data_final.csv")

C:\Users\Vijay\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (3,4,5,6,7,8,9,10,11,12,13,14,16,18,19,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df1['Timecode'] = df1['Timecode'].astype(object)

In [4]:
#df1

In [5]:
#Function that converts the time column into specific time format which is required to break video's for ffmpy library
def tm_res(df1,videoID,Con):
    FMT = '%H:%M:%S'
    newdf = df1[df1.columns[0:4]]                  
    new_df1 = newdf.loc[newdf['ID_val'] == videoID].loc[newdf['Con'] == Con]
    #t1 = new_df1['Time'][new_df1['Time'].index[0]]
    #t2 = new_df1['Time'][new_df1.shape[0]-1]
    t2_init = ''
    t_init=''
    t1 = new_df1['Timecode'].iloc[0]
    t2 = new_df1['Timecode'].iloc[-1]
    t2_splt = str(t2).split('.')
    t1_splt = str(t1).split('.')
    if len(t2_splt) > 1:
        if len(t2_splt[0])==1:
            t2_splt[0] = '0'+t2_splt[0]
        elif t2_splt[0] == '':
            t2_splt[0] = '00'
        if len(t2_splt[1])==1:
            t2_splt[1] = t2_splt[1]+'0'
        t2_init = '00:'+t2_splt[0]+':'+t2_splt[1]
    elif len(t2_splt) == 1:
        if len(t2_splt[0]) == 1:
            t2_splt[0] = '0'+t2_splt[0]+':00'
            t2_init = '00:'+t2_splt[0]
        else:
            t2_init = '00:'+t2_splt[0]+':00'

    if len(t1_splt)>1:
        if len(t1_splt[0])==1:
            t1_splt[0] = '0'+t1_splt[0]
        elif t1_splt[0] == '':
            t1_splt[0] = '00'
        if len(t1_splt[1])==1:
            t1_splt[1] = t1_splt[1]+'0'
        t_init = '00:'+t1_splt[0]+':'+t1_splt[1]
    elif len(t1_splt) == 1:
        if len(t1_splt[0]) == 1:
            t1_splt[0] = '0'+t1_splt[0]+':00'
            t_init = '00:'+t1_splt[0]
        else:
            t_init = '00:'+t1_splt[0]+':00'
            
    tdelta = datetime.strptime(t2_init, FMT) - datetime.strptime(t_init, FMT)
    t = str(tdelta).split(':')
    t_final = '00:' + t[1] + ':' + t[2]
    
    return t_init,t_final

In [6]:
#Location where all the video's are available
data_path = 'C:/Users/Vijay/Isaac/Issac Video/Working_code_on_training_vedios/purl2 video/Session 1'
filenames = [file for file in glob.glob(data_path + '*/*')] 

In [21]:
#code to break the video's according to the interview timing's
#And store it to the specific location
for i in range(len(filenames)):
    out_spt = filenames[i].split('/')[len(filenames[i].split('/'))-1].split('_')[0]
    out_spt = out_spt.split('\\')[len(out_spt.split('\\'))-1]
    if out_spt != 'Session 1':
        out_spt1 = os.path.join(data_path, out_spt)
        print(out_spt1)
        break_path = 'C:/Users/Vijay/Isaac/Issac Video/Working_code_on_training_vedios/purl2 video/final videos/'
        os.mkdir(break_path+out_spt)
        onlyfiles = [f for f in listdir(filenames[i]) if isfile(join(filenames[i], f))]
        fn_nm = filenames[i]+'/'+onlyfiles[0]
        for j in range(1,5):
            output_f = os.path.join(break_path+out_spt, out_spt+'_'+str(j)+'.mov')
            t_init,t_final = tm_res(df1,out_spt,j)
            #print(t_init + "  " + t_final)
            ff = ffmpy.FFmpeg(inputs={fn_nm: None}, outputs={output_f: '-ss %s -t %s -c copy' % (t_init,t_final)})
            #print(fn_nm)
            os.popen(ff.cmd)


C:/Users/Vijay/Isaac/Issac Video/Working_code_on_training_vedios/purl2 video/Session 1\P1441
C:/Users/Vijay/Isaac/Issac Video/Working_code_on_training_vedios/purl2 video/Session 1\P1443
C:/Users/Vijay/Isaac/Issac Video/Working_code_on_training_vedios/purl2 video/Session 1\P1446
C:/Users/Vijay/Isaac/Issac Video/Working_code_on_training_vedios/purl2 video/Session 1\P1452


In [192]:
#command to split video's 
#ffmpeg -i B2019_SSI2.wmv -ss 00:19:28 -t 00:03:34 -c copy out.wmv